In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from PIL import Image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from ultralytics import YOLO

import tensorflow as tf

In [2]:
TRAIN_DATASET_PATH = "./dataset/train/"

In [3]:
path = os.path.join(TRAIN_DATASET_PATH,"labels")

dct = {
    "images": [],
    "labels": [],
    "x_center": [],
    "y_center": [],
    "width": [],
    "height": []
}

for image in os.listdir(path):
    label_file = open(path + "/" + image)

    labels_str = label_file.read().split(sep="\n")
    labels_arr = []
    xmin_arr = []
    ymin_arr = []
    xmax_arr = []
    ymax_arr = []
    for label in labels_str:
        label_parts = label.split()
        labels_arr.append(int(label_parts[0]))
        xmin_arr.append(float(label_parts[1]))
        ymin_arr.append(float(label_parts[2]))
        xmax_arr.append(float(label_parts[3]))
        ymax_arr.append(float(label_parts[4]))

    dct["images"].append(image)
    dct["labels"].append(labels_arr)
    dct["x_center"].append(xmin_arr)
    dct["y_center"].append(ymin_arr)
    dct["width"].append(xmax_arr)
    dct["height"].append(ymax_arr)

    label_file.close()

df = pd.DataFrame(dct)
df

,images,labels,x_center,y_center,width,height
0,009000_jpg.rf.8c46e1aa5b46a0ad24ee4bcb2945d22a...,[2],[0.6538461538461539],[0.5420673076923077],[0.0625],[0.06610576923076923]
1,009002_jpg.rf.18bf80f2cfdb51f853da15019f787cef...,[2],[0.5576923076923077],[0.3389423076923077],[0.055288461538461536],[0.07091346153846154]
2,009003_jpg.rf.46963402c4cb6f46a47e508b892c6521...,[2],[0.4230769230769231],[0.3798076923076923],[0.09975961538461539],[0.055288461538461536]
3,009007_jpg.rf.a5143afbb0c741f3b60fc72403fdde6a...,[2],[0.4014423076923077],[0.35096153846153844],[0.055288461538461536],[0.1778846153846154]
4,009012_jpg.rf.bc99877ade8754d2be89119361e7820c...,[2],[0.8040865384615384],[0.3870192307692308],[0.052884615384615384],[0.07211538461538461]
...,...,...,...,...,...,...
6176,042973_jpg.rf.6792837ba2183435721cd7b4e9674ecd...,[1],[0.3485576923076923],[0.3016826923076923],[0.08533653846153846],[0.08653846153846154]
6177,042982_jpg.rf.79a8b11af76f74faa44312da78ee1486...,[4],[0.3629807692307692],[0.5276442307692307],[0.0625],[0.16706730769230768]
6178,042984_jpg.rf.5005cade401420afa36aac4b818e8dfa...,[1],[0.3870192307692308],[0.36778846153846156],[0.04447115384615385],[0.052884615384615384]
6179,042994_jpg.rf.141176d36edf4a11d33dad95a76e61b6...,[1],[0.5084134615384616],[0.5492788461538461],[0.0889423076923077],[0.06370192307692307]


In [4]:
df[df["labels"].isin([[1]])][:50]

,images,labels,x_center,y_center,width,height
227,009696_jpg.rf.cce06a15455bdfb3ec994de61207f9ec...,[1],[0.7295673076923077],[0.49399038461538464],[0.13822115384615385],[0.0829326923076923]
1272,020535_jpg.rf.1d1f389c1ac9223533b07f19d45d0fed...,[1],[0.5276442307692307],[0.3401442307692308],[0.09254807692307693],[0.12740384615384615]
1986,026032_jpg.rf.7e9a535ddcf0a5013b8c37dffd3739af...,[1],[0.5204326923076923],[0.5072115384615384],[0.06490384615384616],[0.06971153846153846]
1990,026040_jpg.rf.308b8bcf0f012196737af0715b064cbb...,[1],[0.6442307692307693],[0.49399038461538464],[0.06490384615384616],[0.11658653846153846]
1991,026043_jpg.rf.4e6258a0c6557491c1a7619d51f48834...,[1],[0.3245192307692308],[0.5552884615384616],[0.03245192307692308],[0.14783653846153846]
1993,026057_jpg.rf.97d02180613f5ffe322d1cbf89d34031...,[1],[0.5829326923076923],[0.3894230769230769],[0.11177884615384616],[0.07211538461538461]
1994,026074_jpg.rf.7d4a4263d7461b4688a8548deb37ac09...,[1],[0.6129807692307693],[0.5180288461538461],[0.07692307692307693],[0.11899038461538461]
2000,026107_jpg.rf.24622a90c94e31cb97327d1b711882b2...,[1],[0.46033653846153844],[0.5096153846153846],[0.02403846153846154],[0.14543269230769232]
2003,026127_jpg.rf.ddfedfe6d2728271fefebe522bbfaca0...,[1],[0.59375],[0.3293269230769231],[0.06009615384615385],[0.052884615384615384]
2007,026135_jpg.rf.1b7417b83015da5e83efcfb417a00623...,[1],[0.6382211538461539],[0.5120192307692307],[0.06850961538461539],[0.125]


In [4]:
images_path_df = pd.DataFrame({
    "images": os.listdir(TRAIN_DATASET_PATH + "images")
})

images_path_df

,images
0,009000_jpg.rf.8c46e1aa5b46a0ad24ee4bcb2945d22a...
1,009002_jpg.rf.18bf80f2cfdb51f853da15019f787cef...
2,009003_jpg.rf.46963402c4cb6f46a47e508b892c6521...
3,009007_jpg.rf.a5143afbb0c741f3b60fc72403fdde6a...
4,009012_jpg.rf.bc99877ade8754d2be89119361e7820c...
...,...
6176,042973_jpg.rf.6792837ba2183435721cd7b4e9674ecd...
6177,042982_jpg.rf.79a8b11af76f74faa44312da78ee1486...
6178,042984_jpg.rf.5005cade401420afa36aac4b818e8dfa...
6179,042994_jpg.rf.141176d36edf4a11d33dad95a76e61b6...


In [5]:
TRAIN_DATASET_PATH + "images/" + images_path_df["images"][8]

'./dataset/train/images/009017_jpg.rf.366548d28208b7ab6d0d04c133fd5f39.jpg'

In [6]:
plt.imshow(Image.open(TRAIN_DATASET_PATH + "images/" + images_path_df["images"][8]))

: 

In [ ]:
fig, ax = plt.subplots(1)
index = 2110

x_start = df["x_center"][index][0] - df["width"][index][0] / 2
y_start = df["y_center"][index][0] - df["height"][index][0] / 2

# rect = patches.Rectangle((x_start * 416, y_start * 416), df["width"][index][0] * 416, df["height"][index][0] * 416, edgecolor="red", facecolor="none")

# ax.add_patch(rect)
ax.imshow(Image.open(TRAIN_DATASET_PATH + "images/" + images_path_df["images"][index]))
print(df["labels"][index][0])

plt.show()

1


: 